In [1]:
import sqlite3

'''
一共開 3 個 DB
1. 存 NER 結果
2. 存 POS 結果
3. 存 RE 結果: 含網址、Email、Username、(行動、市內)電話、IP
'''

con_NER = sqlite3.connect("JudicialAnalysis_R_NER.db")
cur_NER = con_NER.cursor()

con_POS = sqlite3.connect("JudicialAnalysis_R_POS.db")
cur_POS = con_POS.cursor()

con_RE = sqlite3.connect("JudicialAnalysis_R_RE.db")
cur_RE = con_RE.cursor()

In [2]:
# 創建 Table
new = 0
if new:
    cur_NER.execute("CREATE TABLE results(jid, date, keyword_type, keywords)")
    con_NER.commit()
    con_NER.close()
    
    cur_POS.execute("CREATE TABLE results(jid, date,  keyword_type, keywords)")
    con_POS.commit()
    con_POS.close()
    
    cur_RE.execute("CREATE TABLE results(jid, date, keyword_type, keywords)")
    con_RE.commit()
    con_RE.close()

In [3]:
# 把原本裁判書檔按順序反過來
# 從 2023年12月開始
reverse = 0 # 已經做過就不用再做了
if reverse:
    import os 

    source_file = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files.txt'
            )

    with open(source_file, 'r', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        json_files_R = f.read().strip().split('\n')

    json_files_R.sort(reverse=True)
    json_files_R = [f for f in json_files_R if os.path.isfile(f)]

    source_file_R = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files_R.txt'
            )
    
    json_files_R_STR = '\n'.join(json_files_R).strip()
    
    with open(source_file_R, 'w', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        f.read().strip().split('\n')

In [4]:
import re
from valid_IP import valid_IPv4s, valid_IPv6s

# 用來抓網址s、Domains
def extract_urls(text):
    urls = []
    patterns = [r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', r'https?://([\w\.\-]+)']
    for url_pattern in patterns:
        urls += re.findall(url_pattern, text)
    urls = list(set(urls))
    urls.sort()
    return urls

# 用來抓 Emails
def extract_emails(text):
    email_pattern = r'[\w.-]+@[\w.-]+'
    emails = re.findall(email_pattern, text)
    emails = list(set(emails))
    emails.sort()
    return emails

# 用來抓 Usernames
# \w : 包括(alphanumeric) [0-9a-zA-Z_]
# 後來發現 \w 在 python3 是 unicode 也支持中文
# 解決方法: 要在 re.findall 第三個參數加入 re.A 恢復使用 ASCII，例如: re.findall(r"\w", text, re.A))
# 我是乾脆直接指定 [0-9a-zA-Z_]
def extract_usernames(text):
    # 用戶名模式：開頭可能有@，接著是字母、數字、下劃線、點或破折號
    username_pattern = r'@?[a-zA-Z][0-9a-zA-Z_.-]+'
    usernames = re.findall(username_pattern, text)
    usernames = list(set(usernames))
    usernames.sort()
    return usernames

# 用來抓手機號碼
def extract_mobilephones(text):
    # 手機可能有 - 也可能沒有 -
    # 國際號碼可能有 886
    cellphones = []
    cellphone_patterns = [r'(\+?886\d{3}-?\d{3}-?\d{3})', r'(0\d{3}-?\d{3}-?\d{3})']
    for cellphone_pattern in cellphone_patterns:
        cellphones += re.findall(cellphone_pattern, text)
    cellphones = list(set(cellphones))
    cellphones.sort()
    return cellphones

# 用來抓市內電話號碼
# 規則參考: https://zh.wikipedia.org/zh-tw/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E9%9B%BB%E8%A9%B1%E8%99%9F%E7%A2%BC
def extract_telephones(text):
    telephone_pattern = r'\(?0[2-8]\d{0,3}\)?-?\d{1,4}-?\d{4}'
    telephones = re.findall(telephone_pattern, text)
    telephones = list(set(telephones))
    telephones.sort()
    return telephones

# 用來抓 IP
def extract_ip(text):
    v4a = valid_IPv4s(text)
    v6a = valid_IPv6s(text)
    
    # 結合兩者結果
    ips = list(set(v4a + v6a))
    ips.sort()
    return ips

# 用來初步抓罪名
def extract_crime(text):
    crimes = []
    patterns = [
        r'違反(\S{2,20})等案', 
        r'違反(\S{2,20})案?', 
        r'違反(\S{2,20})等罪', 
        r'違反(\S{2,20})罪',
        r'因(\S{2,20})等案',
        r'因(\S{2,20})案', 
        r'因(\S{2,20})等罪', 
        r'因(\S{2,20})罪', 
        r'號(\S{2,20})等案', 
        r'號(\S{2,20})案?',
        r'號(\S{2,20})等?罪', 
        r'號(\S{2,20})等罪',
        r'號(\S{2,20})罪',
        r'被告(\S{2,20})等案', 
        r'被告(\S{2,20})案', 
        r'被告(\S{2,20})等罪'
        r'被告(\S{2,20})罪'
    ]
    for pattern in patterns:
        crimes += re.findall(pattern, text)
    crimes = [c.replace('違反', '').replace('等', '') for c in crimes if len(c) < 15 and len(c) > 1 ]
    
    crimes = [c for c in crimes if c]
    crimes = [c for c in crimes if not str(c[0]).isdigit()]
    crimes = [c for c in crimes if not '刑事判決' in c and not '聲請' in c and not ',' in c and not '起訴' in c and '刑責' not in c and not '刑責' in c and not '檢察' in c and not '上開' in c and not '事實' in c and not '被告' in c]
    # 檢查是否有包含
    fix_crimes = []
    for c in crimes:
        drop = 0
        for cc in crimes:
            if cc == c:
                continue
            if len(cc) > len(c) and c in cc:
                drop += 1
        if drop == 0:
            fix_crimes.append(cc)
    crimes = fix_crimes + []
    
    # 刪掉奇怪的開頭
    fix_crimes = []
    for c in crimes:
        if len(c) > 2:
            if c[0] == '因':
                c = c[1:]
            if c[:2] == '違反':
                c = c[2:]
        if c:
            fix_crimes.append(c)
    
    crimes = [c for c in fix_crimes if c]
    
    crimes = list(set(crimes))
    crimes.sort()
    
    return crimes

re_extractors = {
    'crimes' : extract_crime,
    'urls' : extract_urls,
    'emails' : extract_emails,
    'usernames' : extract_usernames,
    'mobilephones' : extract_mobilephones,
    'telephones' : extract_telephones,
    'ips' : extract_ip,
}

In [5]:
from ElementExtractor import Judicial_Parser
import os, time, traceback, json, re

In [6]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [7]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
#data_utils.download_data_gdown("./") # gdrive-ckip 已下載就不用再下載

In [8]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)

# 採坑心得
# 新版已經沒有 tensorflow-gpu
# 我使用 python 3.7 的環境 (conda create -n ckipcore python=3.7)
# 先安裝 pip install -U ckiptagger[tf,gdown]
# 再改用 Conda install tensorflow 去安裝匹配的新版
# 搭配 disable_cuda=False 參數，從 windows 工作管理員之效能頁籤看起來，確實有使用 GPU

ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.n

In [9]:
# 簡繁轉換器 (暫時用不到)
# import opencc
# converter = opencc.OpenCC('t2s.json')

In [ ]:
from IPython.display import clear_output

start_time = time.time()
print(f'[現在時間] {time.strftime("%Y-%m-%d %H:%M:%S")}')

source_file = os.path.join(
        'C:\\', '課程', '交大','論文','案件整理',
        'json_files_R.txt'
        )
    
count_file_done_num = 0
clear_output()

# 多工時使用
skip = 0 # 要跳過從第幾開始

print('-'*50)
print(f'>>開始分析...')
print(f'[當前已分析數量] {count_file_done_num}')

with open(source_file, 'r', encoding='utf-8') as f:
    # 2022~2023年判決書有211萬7,454個json檔
    # 故一次讀一個處理，減少記憶體占用
    raw_data = ''
    
    while 1: 
        # 開始逐案讀取資料分析
        file = f.readline()
        count_file_done_num += 1
        
        if '刑事' in file:
            # 篩掉一些比較非刑案的
            if not ('簡易' in file or '民事' in file or '補償' in file or '憲法' in file or '商業' in file):
                # 開始讀取裁判書內容
                file = file.strip()

                # 顯示進度
                temp_time = time.time()
                elapsed_time = temp_time - start_time
                hours = int(elapsed_time // 3600)  # 計算小時數
                minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
                seconds = int(elapsed_time % 60)  # 計算剩餘秒數
                
                clear_output()
                print('-'*50)
                print(f'>>開始分析...')
                print(f'[當前已分析數量] {count_file_done_num}')
                print(f'[累計耗時] {hours}:{minutes}:{seconds}')
                print(f'[當前檔案] {file}')
                
                # 確定 file 存在
                if not os.path.isfile(file):
                    print(f'[並非正常檔案] 跳過...')
                    time.sleep(1)
                    continue
                
                try:
                    with open(file, 'r', encoding='utf-8') as jf:
                        json_data = json.load(jf)
                except:
                    x = traceback.format_exc()
                    print(x)
                    time.sleep(3)
                    continue
                    
                # 資料清洗
                parser = Judicial_Parser(json_data)
                
                jid = parser.jid.strip().replace(' ', '')
                date = parser.dateSTR.strip().replace(' ', '')
                raw = parser.raw
                text = parser.text
                text_u = parser.text_u
                
                # 確定是否已經在資料庫內
                check_res = cur_POS.execute(f"SELECT jid FROM results WHERE jid='{jid}'")
                temp = check_res.fetchone()
                if temp:
                    print(f'[JID] {jid}')
                    print('>>此JID發現資料庫已有，跳過')
                    continue
                    
                print('-'*50)
                print(text_u[:500])
                print('-'*50)
                
                # 沒有在資料庫內才處理
                
                '''
                ------------------------------------------------------------------
                RE 擷取
                ------------------------------------------------------------------
                '''
                 # Input text
                print('[RE] 分析中')
                # 擷取 RE 結果
                re_dict = dict()
                for re_type, re_func in re_extractors.items():
                    re_temp_result = re_func(text_u)
                    if re_temp_result:
                        re_dict[re_type] = re_temp_result
                    print(f'>> [{re_type}] {re_temp_result}')
                
                '''
                ------------------------------------------------------------------
                POS & NER 擷取
                ------------------------------------------------------------------
                '''
                # Input text
                print('-'*50)
                print('[WS] 分析中')
                sentence_list = text_u.split('。')
                word_sentence_list = ws(
                    sentence_list,
                    # sentence_segmentation = True, # To consider delimiters
                    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
                    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                    # coerce_dictionary = dictionary2, # words in this dictionary are forced
                )
                print('[POS] 分析中')
                pos_sentence_list = pos(word_sentence_list)
                print('[NER] 分析中')
                entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
                
                # 整理 POS 結果
                pos_dict = dict()
                for idx, words in enumerate(word_sentence_list):
                    for idx2, word in enumerate(words):
                        temp_type = pos_sentence_list[idx][idx2]
                        word = word.strip().replace(' ', '').replace(':', '').replace(',', '')
                        pos_dict.setdefault(temp_type, set())
                        pos_dict[temp_type].add(word)
                
                # 擷取 NER 結果
                ner_dict = dict()
                for e in entity_sentence_list:
                    for ee in e:
                        ner_type = ee[2]
                        ner_dict.setdefault(ner_type, set())
                        temp_ner_result = ee[3].strip().replace(' ', '').replace(':', '').replace(',', '')
                        ner_dict[ner_type].add(temp_ner_result)
                print('-'*50)
                
                '''
                ------------------------------------------------------------------
                寫入 DB Table
                ------------------------------------------------------------------
                '''
                print('-'*50)
                print('>>準備寫入 RE 結果')
                for re_type, re_keywords in re_dict.items():
                    values_str = ','.join(re_keywords)
                    to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                    cur_RE.execute(to_db_content, (jid, date, re_type, values_str))
                con_RE.commit()
                
                print('-'*50)
                print('>>準備寫入 POS 結果')
                for temp_type, temp_keywords in pos_dict.items():
                    temp_keywords = {tk.strip().replace(' ', '').replace(',', '').replace(':', '') for tk in temp_keywords if len(tk.strip().replace(' ', '').replace(',', '').replace(':', '')) > 1}
                    temp_keywords = list(temp_keywords)
                    temp_keywords.sort()
                    values_str = ','.join(temp_keywords)
                    to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                    cur_POS.execute(to_db_content, (jid, date, re_type, values_str))
                con_POS.commit()
                
                print('-'*50)
                print('>>準備寫入 NER 結果')
                for temp_type, temp_keywords in ner_dict.items():
                    temp_keywords = {tk.strip().replace(' ', '').replace(',', '').replace(':', '') for tk in temp_keywords if len(tk.strip().replace(' ', '').replace(',', '').replace(':', '')) > 1}
                    temp_keywords = list(temp_keywords)
                    temp_keywords.sort()
                    values_str = ','.join(temp_keywords)
                    to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                    cur_NER.execute(to_db_content, (jid, date, re_type, values_str))
                con_NER.commit()
                
                print('-'*50)
                print('>>完成!!!')

                
# 關閉 DB占用
con_RE.close()
con_POS.close()
con_NER.close()

# 顯示統計結果
end_time = time.time()
elapsed_time = end_time - start_time
hours = int(elapsed_time // 3600)  # 計算小時數
minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
seconds = int(elapsed_time % 60)  # 計算剩餘秒數
print(f'[累計耗時] {hours}:{minutes}:{seconds}')

--------------------------------------------------
>>開始分析...
[當前已分析數量] 6713
[累計耗時] 19:53:12
[當前檔案] C:\課程\交大\論文\案件整理\202312\臺灣高等法院臺南分院刑事\TNHM,110,上訴,873,20231228,1.json
--------------------------------------------------
臺灣高等法院臺南分院刑事判決110年度上訴字第873號110年度上訴字第874號112年度上訴字第317號上 訴 人 臺灣嘉義地方檢察署檢察官上 訴 人 
即 被 告 陳雍杰選任辯護人 丁詠純律師 邱皇錡律師 洪千雅律師上 訴 人 
即 被 告 黃正憲選任辯護人 曾錦源律師(法扶律師)被 告 陳美惠選任辯護人 吳碧娟律師被 告 林美如(原名林芯瑩)選任辯護人 林金宗律師(法扶律師)上列上訴人因被告違反貪污治罪條例等案件,不服臺灣嘉義地方法院109年度訴字第402號、第619號中華民國110年5月20日第一審判決(起訴案號:臺灣嘉義地方檢察署108年度偵字第10024號、109年度偵字第91號;追加起訴案號:臺灣嘉義地方檢察署109年度偵字第6821號、第7279號、第7977號);111年度訴字第20號中華民國112年2月9日第一審判決(起訴案號:臺灣嘉義地方檢察署110年度偵字第7145號、第11171號),提起上訴,本院判決如下。主文 臺灣嘉義地方法院109年度訴字第402號、第619號判決關於陳雍杰定應執行刑及緩刑宣告部分,黃正憲有罪之罪刑部分,暨臺灣嘉義地方法院111年度訴字第
--------------------------------------------------
[RE] 分析中
>> [crimes] ['陳雍杰另涉與前']
>> [urls] []
>> [emails] []
>> [usernames] ['A1', 'A4', 'DM', 'DM1000', 'DM10000', 'DM106', 'DM5', 'LINE', 'Line', 'NK00000000', 'PA00000000', 'PR00000000', 'QG00000000', 'TRUSS', 'Truss', 'line']